In [1]:
# import standard modules and DCTR
import os
os.system('for a in /sys/bus/pci/devices/*; do echo 0 | tee -a $a/numa_node>/dev/null; done') # get rid of NUMA node warnings: https://github.com/tensorflow/tensorflow/issues/42738
os.system('nvidia-smi') # make sure nvidia gpu acceleration is working

import sys
import numpy as np
import math
import tensorflow as tf 


tf.compat.v1.logging.set_verbosity('ERROR') # get rid of some warnings that don't impact anything
# make sure GPU usage is enabled
print(tf.config.list_physical_devices('GPU'))

# import DCTR
import DCTR


Wed Mar 20 10:48:15 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.147.05   Driver Version: 525.147.05   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:09:00.0  On |                  N/A |
|  0%   49C    P8    31W / 250W |   7044MiB /  8192MiB |      6%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# load data

In [2]:
data_dir = './Data'


In [3]:
# Load pre-normed Datasets: MiNNLO: X1 | POWHEG hvq: X0
# only contain tt-pair; every event has order: 
# tt-pair, top, anti-top
# every particle has arguments: 
# [pt, y, phi, mass, eta, E, PID, w, theta]
# [0 , 1, 2  , 3   , 4  , 5, 6  , 7, 8    ]


# POWHEG hvq
x0_nrm = []
x0_nrm = DCTR.load_dataset(f'{data_dir}/POWHEG_hvq/13TeV/01-02_normed_converted_lhe.npz', i=3)[:int(1e7)]
print('POWHEG hvq all particles x0_nrm.shape: '+str(x0_nrm.shape))

# plotting data; different from training data
x0_plt = []
x0_plt = DCTR.load_dataset(f'{data_dir}/POWHEG_hvq/13TeV/03-04_converted_lhe.npz', i=3)[:int(3e7)]
print('POWHEG hvq all particles x0_plt.shape: '+str(x0_plt.shape))

x0_plt_nrm = []
x0_plt_nrm = DCTR.load_dataset(f'{data_dir}/POWHEG_hvq/13TeV/03-04_normed_converted_lhe.npz', i=3)[:int(3e7)]
print('POWHEG hvq all particles x0_plt_nrm.shape: '+str(x0_plt_nrm.shape))



POWHEG hvq all particles x0_nrm.shape: (10000000, 3, 9)
POWHEG hvq all particles x0_plt.shape: (30000000, 3, 9)
POWHEG hvq all particles x0_plt_nrm.shape: (30000000, 3, 9)


In [4]:
# MiNNLO
x1_nrm = []
x1_nrm = DCTR.load_dataset(f'{data_dir}/MiNNLO/converted_with_13TeV_NLO/normed_converted_lhe.npz', i=3)
print('MiNNLO all particles x1_nrm.shape: '+str(x1_nrm.shape))

# plotting data
x1_plt = []
x1_plt = DCTR.load_dataset(f'{data_dir}/MiNNLO/converted_with_13TeV_NLO/converted_lhe.npz', i=3)
print('MiNNLO all particles x1_plt.shape: '+str(x1_plt.shape))


MiNNLO all particles x1_nrm.shape: (9553938, 3, 9)
MiNNLO all particles x1_plt.shape: (9553938, 3, 9)


In [5]:
# get normalized event generator weights
x0_wgt = x0_nrm[:, 0, 7].copy()
x0_wgt /= np.mean(x0_wgt) # adjust so mean is 1

x0_plt_wgt = x0_plt_nrm[:, 0, 7].copy()
x0_plt_wgt /= np.mean(x0_plt_wgt) 

x1_wgt = x1_nrm[:, 0, 7].copy()
x1_wgt /= np.mean(x1_wgt)


In [6]:
# delete eta (pseudorapidity) and Energy -> Train only with [pt, y, phi, m, PID]

# delete energy
x0_nrm = np.delete(x0_nrm, 5, -1)
x0_plt_nrm = np.delete(x0_plt_nrm, 5, -1)
x1_nrm = np.delete(x1_nrm, 5, -1)

# delete eta
x0_nrm = np.delete(x0_nrm, 4, -1)
x0_plt_nrm = np.delete(x0_plt_nrm, 4, -1)
x1_nrm = np.delete(x1_nrm, 4, -1)



In [7]:
# prep arrays for training
x_train, x_val, y_train, y_val, wgt_train, wgt_val = DCTR.prep_arrays(x0_nrm, x1_nrm, val=0.25)

# bring into shape for training loop
train_data = (x_train, y_train, x_val, y_val, wgt_train, wgt_val)
plt_data = (x0_plt , x0_plt_nrm, x1_plt, x1_wgt)


2024-03-20 10:53:43.176483: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-20 10:53:43.832776: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 425 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 SUPER, pci bus id: 0000:09:00.0, compute capability: 7.5
2024-03-20 10:53:43.863020: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1759854480 exceeds 10% of free system memory.


# training loop

In [9]:
# second session

K.clear_session()
'''
train_data, plt_data, model=None, lowest_chi2 = 1e6, train_dir = '/tf/home/gdrive/_STUDIUM_/DCTR_Paper/train',
batch_sizes=[4*8192, 8*8192, 16*8192, 32*8192], repeat=5, super_epochs=35, epochs = 5, starting_super_epoch = 0, 
input_dim=5, Phi_sizes = (100,100,128), F_sizes = (128,100,100), loss = 'mse', dropout=0.0, l2_reg=0.0, 
Phi_acts=('linear', 'elu', 'gelu'), F_acts=('gelu', 'gelu', 'linear'), output_act='sigmoid', learning_rate=0.001
'''
# train_loop(train_data, plt_data, batch_sizes=[8*8192, 16*8192, 32*8192], repeat=6, super_epochs=30, 
           train_dir = '/tf/home/gdrive/_STUDIUM_/DCTR_Paper/train_20240310', epochs=6, dropout=0.05, learning_rate=0.002)

# chi2 is actually reduced chi2 -> 1 would be optimal



starting super_epoch 0

starting training with batch_size: 65536 and 6 epochs
starting with weights from model: None
starting run 0 of super_epoch 0 with batch_size 65536


2024-03-10 09:02:25.423240: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6212 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 SUPER, pci bus id: 0000:09:00.0, compute capability: 7.5
2024-03-10 09:02:37.106260: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.



 best loss 0.2119 of run 0 of super_epoch 0 with batch_size 65536

starting run 1 of super_epoch 0 with batch_size 65536


2024-03-10 09:03:23.425631: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6212 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 SUPER, pci bus id: 0000:09:00.0, compute capability: 7.5



Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0012000000569969416.

 best loss 0.2120 of run 1 of super_epoch 0 with batch_size 65536

starting run 2 of super_epoch 0 with batch_size 65536


...

2024-03-10 09:27:38.515421: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6212 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 SUPER, pci bus id: 0000:09:00.0, compute capability: 7.5



 best loss 0.2126 of run 4 of super_epoch 0 with batch_size 262144

starting run 5 of super_epoch 0 with batch_size 262144


2024-03-10 09:28:35.833038: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6212 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 SUPER, pci bus id: 0000:09:00.0, compute capability: 7.5



 best loss 0.2129 of run 5 of super_epoch 0 with batch_size 262144

calculating stats for 6 models

finished 6 runs of batch_size 262144
in super epoch 0
with best model /tf/home/gdrive/_STUDIUM_/DCTR_Paper/train_20240310/super_epoch_0/run_0/s-0_b-262144_r-0.tf
with chi2 7.2374 and loss 0.2126


finished super_epoch 0 with 6 runs each with batch_sizes:[65536, 131072, 262144]
best model/tf/home/gdrive/_STUDIUM_/DCTR_Paper/train_20240310/super_epoch_0/run_4/s-0_b-131072_r-4.tfwith chi2 4.9728 and loss 0.2121
starting super_epoch 1

starting training with batch_size: 65536 and 6 epochs
starting with weights from model: /tf/home/gdrive/_STUDIUM_/DCTR_Paper/train_20240310/super_epoch_0/run_4/s-0_b-131072_r-4.tf
starting run 0 of super_epoch 1 with batch_size 65536

Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0012000000569969416.

 best loss 0.2117 of run 0 of super_epoch 1 with batch_size 65536

starting run 1 of super_epoch 1 with batch_size 65536

Epoch 5: ReduceLROnPlateau re

NameError: name 'lowest_chi2_list' is not defined

In [11]:
# second session continuation

K.clear_session()
'''
train_data, plt_data, model=None, lowest_chi2 = 1e6, train_dir = '/tf/home/gdrive/_STUDIUM_/DCTR_Paper/train',
batch_sizes=[4*8192, 8*8192, 16*8192, 32*8192], repeat=5, super_epochs=35, epochs = 5, starting_super_epoch = 0, 
input_dim=5, Phi_sizes = (100,100,128), F_sizes = (128,100,100), loss = 'mse', dropout=0.0, l2_reg=0.0, 
Phi_acts=('linear', 'elu', 'gelu'), F_acts=('gelu', 'gelu', 'linear'), output_act='sigmoid', learning_rate=0.001
'''
# train_loop(train_data, plt_data, batch_sizes=[2*8192, 4*8192, 8*8192, 16*8192, 32*8192], repeat=10, super_epochs=10, starting_super_epoch = 30,
           model = '/tf/home/gdrive/_STUDIUM_/DCTR_Paper/train_20240310/super_epoch_4/run_5/s-4_b-131072_r-5.tf',
           train_dir = '/tf/home/gdrive/_STUDIUM_/DCTR_Paper/train_20240310', epochs=10, dropout=0.1, learning_rate=0.001)

# chi2 is actually reduced chi2 -> 1 would be optimal



starting super_epoch 30

starting training with batch_size: 16384 and 10 epochs
starting with weights from model: /tf/home/gdrive/_STUDIUM_/DCTR_Paper/train_20240310/super_epoch_4/run_5/s-4_b-131072_r-5.tf
starting run 0 of super_epoch 30 with batch_size 16384

Epoch 5: ReduceLROnPlateau reducing learning rate to 3.359232141519897e-05.

Epoch 9: ReduceLROnPlateau reducing learning rate to 2.015539284911938e-05.

 best loss 0.2115 of run 0 of super_epoch 30 with batch_size 16384

starting run 1 of super_epoch 30 with batch_size 16384

Epoch 5: ReduceLROnPlateau reducing learning rate to 3.359232141519897e-05.

Epoch 9: ReduceLROnPlateau reducing learning rate to 2.015539284911938e-05.

 best loss 0.2115 of run 1 of super_epoch 30 with batch_size 16384

starting run 2 of super_epoch 30 with batch_size 16384

Epoch 5: ReduceLROnPlateau reducing learning rate to 3.359232141519897e-05.

Epoch 9: ReduceLROnPlateau reducing learning rate to 2.015539284911938e-05.

 best loss 0.2115 of run 2 o

KeyboardInterrupt: 

In [8]:
train_loop(train_data, plt_data, batch_sizes=[8192, 2*8192, 4*8192, 8*8192], repeat=10, super_epochs=15, starting_super_epoch = 33, lowest_chi2 = 3,
           model = '/tf/home/gdrive/_STUDIUM_/DCTR_Paper/train_20240310/super_epoch_30/run_6/s-30_b-16384_r-6.tf',
           train_dir = '/tf/home/gdrive/_STUDIUM_/DCTR_Paper/train_20240310', epochs=15, dropout=0.15, learning_rate=0.0005)


starting super_epoch 33

starting training with batch_size: 8192 and 15 epochs
starting with weights from model: /tf/home/gdrive/_STUDIUM_/DCTR_Paper/train_20240310/super_epoch_30/run_6/s-30_b-16384_r-6.tf
starting run 0 of super_epoch 33 with batch_size 8192


2024-03-11 09:07:08.094500: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.



Epoch 7: ReduceLROnPlateau reducing learning rate to 1.209323527291417e-05.

Epoch 13: ReduceLROnPlateau reducing learning rate to 7.041496064630337e-06.

 best loss 0.2115 of run 0 of super_epoch 33 with batch_size 8192

starting run 1 of super_epoch 33 with batch_size 8192

Epoch 7: ReduceLROnPlateau reducing learning rate to 1.209323527291417e-05.

Epoch 13: ReduceLROnPlateau reducing learning rate to 7.041496064630337e-06.

 best loss 0.2115 of run 1 of super_epoch 33 with batch_size 8192

starting run 2 of super_epoch 33 with batch_size 8192

Epoch 7: ReduceLROnPlateau reducing learning rate to 1.209323527291417e-05.

Epoch 13: ReduceLROnPlateau reducing learning rate to 7.041496064630337e-06.

 best loss 0.2115 of run 2 of super_epoch 33 with batch_size 8192

starting run 3 of super_epoch 33 with batch_size 8192

Epoch 7: ReduceLROnPlateau reducing learning rate to 1.209323527291417e-05.

Epoch 13: ReduceLROnPlateau reducing learning rate to 7.041496064630337e-06.

 best loss 0.

2024-03-11 09:34:45.703244: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1800000000 exceeds 10% of free system memory.
2024-03-11 09:34:46.084054: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1800000000 exceeds 10% of free system memory.
2024-03-11 09:34:52.996159: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1800000000 exceeds 10% of free system memory.
2024-03-11 09:34:53.377724: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1800000000 exceeds 10% of free system memory.


calculating stats for 10 models

finished 10 runs of batch_size 8192
in super epoch 33
with best model /tf/home/gdrive/_STUDIUM_/DCTR_Paper/train_20240310/super_epoch_33/run_2/s-33_b-8192_r-2.tf
with chi2 2.2136 and loss 0.2115
starting training with batch_size: 16384 and 15 epochs
starting with weights from model: /tf/home/gdrive/_STUDIUM_/DCTR_Paper/train_20240310/super_epoch_30/run_6/s-30_b-16384_r-6.tf
starting run 0 of super_epoch 33 with batch_size 16384

Epoch 7: ReduceLROnPlateau reducing learning rate to 1.209323527291417e-05.

Epoch 13: ReduceLROnPlateau reducing learning rate to 7.041496064630337e-06.

 best loss 0.2115 of run 0 of super_epoch 33 with batch_size 16384

starting run 1 of super_epoch 33 with batch_size 16384

Epoch 7: ReduceLROnPlateau reducing learning rate to 1.209323527291417e-05.

Epoch 13: ReduceLROnPlateau reducing learning rate to 7.041496064630337e-06.

 best loss 0.2115 of run 1 of super_epoch 33 with batch_size 16384

starting run 2 of super_epoch 33

KeyboardInterrupt: 

In [9]:
train_loop(train_data, plt_data, batch_sizes=[4096, 8192], repeat=6, super_epochs=7, starting_super_epoch = 37, lowest_chi2 = 2.1,
           model = '/tf/home/gdrive/_STUDIUM_/DCTR_Paper/train_20240310/super_epoch_30/run_6/s-30_b-16384_r-6.tf', 
           train_dir = '/tf/home/gdrive/_STUDIUM_/DCTR_Paper/train_20240310', epochs=8, dropout=0.02, learning_rate=0.0001)


starting super_epoch 37

starting training with batch_size: 4096 and 8 epochs
starting with weights from model: /tf/home/gdrive/_STUDIUM_/DCTR_Paper/train_20240310/super_epoch_30/run_6/s-30_b-16384_r-6.tf
starting run 0 of super_epoch 37 with batch_size 4096


2024-03-11 14:11:28.774413: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.



Epoch 4: ReduceLROnPlateau reducing learning rate to 1.209323527291417e-05.

Epoch 7: ReduceLROnPlateau reducing learning rate to 7.255941272887867e-06.

 best loss 0.2116 of run 0 of super_epoch 37 with batch_size 4096

starting run 1 of super_epoch 37 with batch_size 4096

Epoch 4: ReduceLROnPlateau reducing learning rate to 1.209323527291417e-05.

Epoch 7: ReduceLROnPlateau reducing learning rate to 7.255941272887867e-06.

 best loss 0.2116 of run 1 of super_epoch 37 with batch_size 4096

starting run 2 of super_epoch 37 with batch_size 4096

Epoch 4: ReduceLROnPlateau reducing learning rate to 1.209323527291417e-05.

Epoch 7: ReduceLROnPlateau reducing learning rate to 7.255941272887867e-06.

 best loss 0.2116 of run 2 of super_epoch 37 with batch_size 4096

starting run 3 of super_epoch 37 with batch_size 4096

Epoch 4: ReduceLROnPlateau reducing learning rate to 1.209323527291417e-05.

Epoch 7: ReduceLROnPlateau reducing learning rate to 7.255941272887867e-06.

 best loss 0.2116

2024-03-11 14:22:40.296996: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1800000000 exceeds 10% of free system memory.
2024-03-11 14:22:41.017236: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1800000000 exceeds 10% of free system memory.
2024-03-11 14:22:48.438809: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1800000000 exceeds 10% of free system memory.
2024-03-11 14:22:49.287432: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1800000000 exceeds 10% of free system memory.


calculating stats for 6 models

finished 6 runs of batch_size 4096
in super epoch 37
with best model /tf/home/gdrive/_STUDIUM_/DCTR_Paper/train_20240310/super_epoch_37/run_1/s-37_b-4096_r-1.tf
with chi2 2.0931 and loss 0.2116
starting training with batch_size: 8192 and 8 epochs
starting with weights from model: /tf/home/gdrive/_STUDIUM_/DCTR_Paper/train_20240310/super_epoch_30/run_6/s-30_b-16384_r-6.tf
starting run 0 of super_epoch 37 with batch_size 8192

Epoch 4: ReduceLROnPlateau reducing learning rate to 1.209323527291417e-05.

Epoch 7: ReduceLROnPlateau reducing learning rate to 7.255941272887867e-06.

 best loss 0.2116 of run 0 of super_epoch 37 with batch_size 8192

starting run 1 of super_epoch 37 with batch_size 8192

Epoch 4: ReduceLROnPlateau reducing learning rate to 1.209323527291417e-05.

Epoch 7: ReduceLROnPlateau reducing learning rate to 7.255941272887867e-06.

 best loss 0.2116 of run 1 of super_epoch 37 with batch_size 8192

starting run 2 of super_epoch 37 with batc

(['/tf/home/gdrive/_STUDIUM_/DCTR_Paper/train_20240310/super_epoch_37/run_1/s-37_b-4096_r-1.tf'],
 [2.0930774819210742],
 [0.21157675981521606])

In [ ]:
# best models
# 2.0807: /tf/home/gdrive/_STUDIUM_/DCTR_Paper/train_20240310/super_epoch_30/run_6/s-30_b-16384_r-6.tf
# 2.0931: /tf/home/gdrive/_STUDIUM_/DCTR_Paper/train_20240310/super_epoch_37/run_1/s-37_b-4096_r-1.tf


In [35]:
# first session

K.clear_session()
'''
train_data, plt_data, model=None, lowest_chi2 = 1e6, train_dir = '/tf/home/gdrive/_STUDIUM_/DCTR_Paper/train',
batch_sizes=[4*8192, 8*8192, 16*8192, 32*8192], repeat=5, super_epochs=35, epochs = 5, starting_super_epoch = 0, 
input_dim=5, Phi_sizes = (100,100,128), F_sizes = (128,100,100), loss = 'mse', dropout=0.0, l2_reg=0.0, 
Phi_acts=('linear', 'elu', 'gelu'), F_acts=('gelu', 'gelu', 'linear'), output_act='sigmoid', learning_rate=0.001
'''
# train_loop(train_data, plt_data, batch_sizes=[4*8192, 8*8192, 16*8192, 32*8192], repeat=5, super_epochs=15, epochs=5)

# chi2 is actually reduced chi2 -> 1 would be optimal

starting super_epoch 0

starting training with batch_size: 32768 and 5 epochs
starting run 0 of super_epoch 0 with batch_size 32768


2024-03-06 14:18:28.675571: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6153 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 SUPER, pci bus id: 0000:09:00.0, compute capability: 7.5



 best loss 0.2119 of run 0 of super_epoch 0 with batch_size 32768

starting run 1 of super_epoch 0 with batch_size 32768

 best loss 0.2119 of run 1 of super_epoch 0 with batch_size 32768

starting run 2 of super_epoch 0 with batch_size 32768

 best loss 0.2120 of run 2 of super_epoch 0 with batch_size 32768

starting run 3 of super_epoch 0 with batch_size 32768

 best loss 0.2120 of run 3 of super_epoch 0 with batch_size 32768

starting run 4 of super_epoch 0 with batch_size 32768

 best loss 0.2119 of run 4 of super_epoch 0 with batch_size 32768

calculating stats for 5 models


/tf/home/gdrive/_STUDIUM_/DCTR_Paper/DCTR.py:829: RuntimeWarning: invalid value encountered in true_divide
  chi2 = np.nansum(np.power(n_list[0] - n, 2)/(np.power(uncert, 2) + np.power(uncert_list[0], 2)))



 finished 5 runs of batch_size 32768
    in super epoch 0
   with best model /tf/home/gdrive/_STUDIUM_/DCTR_Paper/train/super_epoch_0/run_1/s-0_b-32768_r-1.tf
    with chi2 12.3689 and loss 0.2119
starting training with batch_size: 65536 and 5 epochs
starting run 0 of super_epoch 0 with batch_size 65536

 best loss 0.2123 of run 0 of super_epoch 0 with batch_size 65536

starting run 1 of super_epoch 0 with batch_size 65536

 best loss 0.2121 of run 1 of super_epoch 0 with batch_size 65536

starting run 2 of super_epoch 0 with batch_size 65536

 best loss 0.2121 of run 2 of super_epoch 0 with batch_size 65536

starting run 3 of super_epoch 0 with batch_size 65536

 best loss 0.2122 of run 3 of super_epoch 0 with batch_size 65536

starting run 4 of super_epoch 0 with batch_size 65536

 best loss 0.2121 of run 4 of super_epoch 0 with batch_size 65536

calculating stats for 5 models


...

/tf/home/gdrive/_STUDIUM_/DCTR_Paper/DCTR.py:829: RuntimeWarning: invalid value encountered in true_divide
  chi2 = np.nansum(np.power(n_list[0] - n, 2)/(np.power(uncert, 2) + np.power(uncert_list[0], 2)))



 finished 5 runs of batch_size 131072
    in super epoch 14
   with best model /tf/home/gdrive/_STUDIUM_/DCTR_Paper/train/super_epoch_14/run_2/s-14_b-131072_r-2.tf
    with chi2 3.7980 and loss 0.2115
starting training with batch_size: 262144 and 5 epochs
starting run 0 of super_epoch 14 with batch_size 262144

Epoch 3: ReduceLROnPlateau reducing learning rate to 1.6796160707599483e-05.

Epoch 5: ReduceLROnPlateau reducing learning rate to 1.007769642455969e-05.

 best loss 0.2115 of run 0 of super_epoch 14 with batch_size 262144

starting run 1 of super_epoch 14 with batch_size 262144

Epoch 3: ReduceLROnPlateau reducing learning rate to 1.6796160707599483e-05.

Epoch 5: ReduceLROnPlateau reducing learning rate to 1.007769642455969e-05.

 best loss 0.2115 of run 1 of super_epoch 14 with batch_size 262144

starting run 2 of super_epoch 14 with batch_size 262144

Epoch 3: ReduceLROnPlateau reducing learning rate to 1.6796160707599483e-05.

Epoch 5: ReduceLROnPlateau reducing learning r

/tf/home/gdrive/_STUDIUM_/DCTR_Paper/DCTR.py:829: RuntimeWarning: invalid value encountered in true_divide
  chi2 = np.nansum(np.power(n_list[0] - n, 2)/(np.power(uncert, 2) + np.power(uncert_list[0], 2)))



 finished 5 runs of batch_size 262144
    in super epoch 14
   with best model /tf/home/gdrive/_STUDIUM_/DCTR_Paper/train/super_epoch_14/run_4/s-14_b-262144_r-4.tf
    with chi2 3.8243 and loss 0.2115
no improvement, lowering learnng_rate to {learning_rate}


 finished super_epoch 14 with 5 runs each with batch_sizes:[32768, 65536, 131072, 262144]
   best model/tf/home/gdrive/_STUDIUM_/DCTR_Paper/train/super_epoch_4/run_0/s-4_b-65536_r-0.tf
    with chi2 2.9960 and loss 0.2116



 finished loop of 15 super_epochs
    with batch_sizes:[32768, 65536, 131072, 262144]
    best model/tf/home/gdrive/_STUDIUM_/DCTR_Paper/train/super_epoch_4/run_0/s-4_b-65536_r-0.tf
    with chi2 2.9960 and loss 0.2116


NameError: name 'lowest_chi2_list' is not defined

In [8]:
# continue training with best weughts from first session as baseline

K.clear_session()
# train_loop(train_data, plt_data, train_dir = '/tf/home/gdrive/_STUDIUM_/DCTR_Paper/train_d10',model='/tf/home/gdrive/_STUDIUM_/DCTR_Paper/train/super_epoch_4/run_0/s-4_b-65536_r-0.tf', 
           lowest_chi2 = 5, repeat=4, super_epochs=35, starting_super_epoch=16, super_patience = 3, learning_rate=0.0005, dropout = 0.1, 
           batch_sizes=[4096, 8192, 2*8192, 4*8192, 8*8192, 16*8192])




starting super_epoch 16

starting training with batch_size: 4096 and 8 epochs
starting run 0 of super_epoch 16 with batch_size 4096


2024-03-07 14:29:25.603131: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.



Epoch 4: ReduceLROnPlateau reducing learning rate to 1.6796160707599483e-05.

Epoch 7: ReduceLROnPlateau reducing learning rate to 1.007769642455969e-05.

 best loss 0.2116 of run 0 of super_epoch 16 with batch_size 4096

starting run 1 of super_epoch 16 with batch_size 4096

Epoch 4: ReduceLROnPlateau reducing learning rate to 1.6796160707599483e-05.

Epoch 7: ReduceLROnPlateau reducing learning rate to 1.007769642455969e-05.

 best loss 0.2116 of run 1 of super_epoch 16 with batch_size 4096

starting run 2 of super_epoch 16 with batch_size 4096

Epoch 4: ReduceLROnPlateau reducing learning rate to 1.6796160707599483e-05.

Epoch 7: ReduceLROnPlateau reducing learning rate to 1.007769642455969e-05.

 best loss 0.2116 of run 2 of super_epoch 16 with batch_size 4096

starting run 3 of super_epoch 16 with batch_size 4096

Epoch 4: ReduceLROnPlateau reducing learning rate to 1.6796160707599483e-05.

Epoch 7: ReduceLROnPlateau reducing learning rate to 1.007769642455969e-05.

 best loss 0.

2024-03-07 14:36:24.528930: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1800000000 exceeds 10% of free system memory.
2024-03-07 14:36:24.910526: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1800000000 exceeds 10% of free system memory.
2024-03-07 14:36:31.871218: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1800000000 exceeds 10% of free system memory.
2024-03-07 14:36:32.237766: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1800000000 exceeds 10% of free system memory.


calculating stats for 4 models


...


 finished 4 runs of batch_size 131072
    in super epoch 19
   with best model /tf/home/gdrive/_STUDIUM_/DCTR_Paper/train_d10/super_epoch_19/run_0/s-19_b-131072_r-0.tf
    with chi2 1.9494 and loss 0.2116
no improvement, lowering learnng_rate to {learning_rate}


 finished super_epoch 19 with 4 runs each with batch_sizes:[4096, 8192, 16384, 32768, 65536, 131072]
   best model/tf/home/gdrive/_STUDIUM_/DCTR_Paper/train_d10/super_epoch_16/run_2/s-16_b-65536_r-2.tf
    with chi2 1.5110 and loss 0.2116
starting super_epoch 20

starting training with batch_size: 4096 and 8 epochs
starting run 0 of super_epoch 20 with batch_size 4096

Epoch 4: ReduceLROnPlateau reducing learning rate to 6.046617636457085e-06.

Epoch 7: ReduceLROnPlateau reducing learning rate to 3.6279706364439334e-06.

 best loss 0.2116 of run 0 of super_epoch 20 with batch_size 4096

starting run 1 of super_epoch 20 with batch_size 4096

Epoch 4: ReduceLROnPlateau reducing learning rate to 6.046617636457085e-06.

Epoch 7: 

/tf/home/gdrive/_STUDIUM_/DCTR_Paper/DCTR.py:829: RuntimeWarning: invalid value encountered in true_divide
  chi2 = np.nansum(np.power(n_list[0] - n, 2)/(np.power(uncert, 2) + np.power(uncert_list[0], 2)))



 finished 4 runs of batch_size 4096
    in super epoch 20
   with best model /tf/home/gdrive/_STUDIUM_/DCTR_Paper/train_d10/super_epoch_20/run_1/s-20_b-4096_r-1.tf
    with chi2 2.0994 and loss 0.2116
starting training with batch_size: 8192 and 8 epochs
starting run 0 of super_epoch 20 with batch_size 8192

Epoch 4: ReduceLROnPlateau reducing learning rate to 6.046617636457085e-06.

Epoch 7: ReduceLROnPlateau reducing learning rate to 3.6279706364439334e-06.

 best loss 0.2116 of run 0 of super_epoch 20 with batch_size 8192

starting run 1 of super_epoch 20 with batch_size 8192

Epoch 4: ReduceLROnPlateau reducing learning rate to 6.046617636457085e-06.

Epoch 7: ReduceLROnPlateau reducing learning rate to 3.6279706364439334e-06.

 best loss 0.2116 of run 1 of super_epoch 20 with batch_size 8192

starting run 2 of super_epoch 20 with batch_size 8192

Epoch 4: ReduceLROnPlateau reducing learning rate to 6.046617636457085e-06.

Epoch 7: ReduceLROnPlateau reducing learning rate to 3.6279

2024-03-07 18:10:27.295449: W tensorflow/core/common_runtime/bfc_allocator.cc:462] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.68GiB (rounded to 1800000000)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-03-07 18:10:27.299047: I tensorflow/core/common_runtime/bfc_allocator.cc:1010] BFCAllocator dump for GPU_0_bfc
2024-03-07 18:10:27.299066: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (256): 	Total Chunks: 103, Chunks in use: 103. 25.8KiB allocated for chunks. 25.8KiB in use in bin. 508B client-requested in use in bin.
2024-03-07 18:10:27.299071: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (512): 	Total Chunks: 43, Chunks in use: 42. 22.2KiB allocated for chunks. 21.8KiB in use in bin. 17.9KiB client-requested in use in bin.
2024-03-07 18:10:27.299075

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [12]:
# /tf/home/gdrive/_STUDIUM_/DCTR_Paper/train_d10/super_epoch_16/run_2/s-16_b-65536_r-2.tf

train_loop(train_data, plt_data, train_dir = '/tf/home/gdrive/_STUDIUM_/DCTR_Paper/train_d20',model='/tf/home/gdrive/_STUDIUM_/DCTR_Paper/train_d10/super_epoch_16/run_2/s-16_b-65536_r-2.tf', 
           lowest_chi2 = 3, repeat=4, super_epochs=35, starting_super_epoch=20, super_patience = 3, learning_rate=0.0005, dropout = 0.2, 
           batch_sizes=[6*8192, 8*8192, 12*8192])



starting super_epoch 20

starting training with batch_size: 49152 and 8 epochs
starting run 0 of super_epoch 20 with batch_size 49152

Epoch 4: ReduceLROnPlateau reducing learning rate to 6.046617636457085e-06.

Epoch 7: ReduceLROnPlateau reducing learning rate to 3.6279706364439334e-06.

 best loss 0.2116 of run 0 of super_epoch 20 with batch_size 49152

starting run 1 of super_epoch 20 with batch_size 49152

Epoch 4: ReduceLROnPlateau reducing learning rate to 6.046617636457085e-06.

Epoch 7: ReduceLROnPlateau reducing learning rate to 3.6279706364439334e-06.

 best loss 0.2116 of run 1 of super_epoch 20 with batch_size 49152

starting run 2 of super_epoch 20 with batch_size 49152

Epoch 4: ReduceLROnPlateau reducing learning rate to 6.046617636457085e-06.

Epoch 7: ReduceLROnPlateau reducing learning rate to 3.6279706364439334e-06.

 best loss 0.2116 of run 2 of super_epoch 20 with batch_size 49152

starting run 3 of super_epoch 20 with batch_size 49152

Epoch 4: ReduceLROnPlateau r

/tf/home/gdrive/_STUDIUM_/DCTR_Paper/DCTR.py:829: RuntimeWarning: invalid value encountered in true_divide
  chi2 = np.nansum(np.power(n_list[0] - n, 2)/(np.power(uncert, 2) + np.power(uncert_list[0], 2)))



 finished 4 runs of batch_size 49152
    in super epoch 20
   with best model /tf/home/gdrive/_STUDIUM_/DCTR_Paper/train_d20/super_epoch_20/run_1/s-20_b-49152_r-1.tf
    with chi2 2.0278 and loss 0.2116
starting training with batch_size: 65536 and 8 epochs
starting run 0 of super_epoch 20 with batch_size 65536

Epoch 4: ReduceLROnPlateau reducing learning rate to 6.046617636457085e-06.

Epoch 7: ReduceLROnPlateau reducing learning rate to 3.6279706364439334e-06.

 best loss 0.2116 of run 0 of super_epoch 20 with batch_size 65536

starting run 1 of super_epoch 20 with batch_size 65536

Epoch 4: ReduceLROnPlateau reducing learning rate to 6.046617636457085e-06.

Epoch 7: ReduceLROnPlateau reducing learning rate to 3.6279706364439334e-06.

 best loss 0.2116 of run 1 of super_epoch 20 with batch_size 65536

starting run 2 of super_epoch 20 with batch_size 65536

Epoch 4: ReduceLROnPlateau reducing learning rate to 6.046617636457085e-06.

Epoch 7: ReduceLROnPlateau reducing learning rate t

/tf/home/gdrive/_STUDIUM_/DCTR_Paper/DCTR.py:829: RuntimeWarning: invalid value encountered in true_divide
  chi2 = np.nansum(np.power(n_list[0] - n, 2)/(np.power(uncert, 2) + np.power(uncert_list[0], 2)))



 finished 4 runs of batch_size 65536
    in super epoch 20
   with best model /tf/home/gdrive/_STUDIUM_/DCTR_Paper/train_d20/super_epoch_20/run_0/s-20_b-65536_r-0.tf
    with chi2 1.8601 and loss 0.2116
starting training with batch_size: 98304 and 8 epochs
starting run 0 of super_epoch 20 with batch_size 98304

Epoch 4: ReduceLROnPlateau reducing learning rate to 6.046617636457085e-06.

Epoch 7: ReduceLROnPlateau reducing learning rate to 3.6279706364439334e-06.

 best loss 0.2116 of run 0 of super_epoch 20 with batch_size 98304

starting run 1 of super_epoch 20 with batch_size 98304

Epoch 4: ReduceLROnPlateau reducing learning rate to 6.046617636457085e-06.

Epoch 7: ReduceLROnPlateau reducing learning rate to 3.6279706364439334e-06.

 best loss 0.2116 of run 1 of super_epoch 20 with batch_size 98304

starting run 2 of super_epoch 20 with batch_size 98304

Epoch 4: ReduceLROnPlateau reducing learning rate to 6.046617636457085e-06.

Epoch 7: ReduceLROnPlateau reducing learning rate t

...


 finished 4 runs of batch_size 98304
    in super epoch 21
   with best model /tf/home/gdrive/_STUDIUM_/DCTR_Paper/train_d20/super_epoch_21/run_3/s-21_b-98304_r-3.tf
    with chi2 2.0604 and loss 0.2116
no improvement, lowering learnng_rate to {learning_rate}


 finished super_epoch 21 with 4 runs each with batch_sizes:[49152, 65536, 98304]
   best model/tf/home/gdrive/_STUDIUM_/DCTR_Paper/train_d20/super_epoch_20/run_0/s-20_b-65536_r-0.tf
    with chi2 1.8601 and loss 0.2116
starting super_epoch 22

starting training with batch_size: 49152 and 8 epochs
starting run 0 of super_epoch 22 with batch_size 49152

Epoch 4: ReduceLROnPlateau reducing learning rate to 2.1767824364360423e-06.

Epoch 7: ReduceLROnPlateau reducing learning rate to 1.3060694072919432e-06.

 best loss 0.2116 of run 0 of super_epoch 22 with batch_size 49152

starting run 1 of super_epoch 22 with batch_size 49152

Epoch 4: ReduceLROnPlateau reducing learning rate to 2.1767824364360423e-06.

Epoch 7: ReduceLROnPlatea

/tf/home/gdrive/_STUDIUM_/DCTR_Paper/DCTR.py:829: RuntimeWarning: invalid value encountered in true_divide
  chi2 = np.nansum(np.power(n_list[0] - n, 2)/(np.power(uncert, 2) + np.power(uncert_list[0], 2)))



 finished 4 runs of batch_size 49152
    in super epoch 22
   with best model /tf/home/gdrive/_STUDIUM_/DCTR_Paper/train_d20/super_epoch_22/run_2/s-22_b-49152_r-2.tf
    with chi2 1.9880 and loss 0.2116
starting training with batch_size: 65536 and 8 epochs
starting run 0 of super_epoch 22 with batch_size 65536


2024-03-07 20:07:42.341349: W tensorflow/core/common_runtime/bfc_allocator.cc:462] Allocator (GPU_0_bfc) ran out of memory trying to allocate 25.00MiB (rounded to 26214400)requested by op gradient_tape/model_8/output/MatMul/MatMul
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-03-07 20:07:42.341415: I tensorflow/core/common_runtime/bfc_allocator.cc:1010] BFCAllocator dump for GPU_0_bfc
2024-03-07 20:07:42.341430: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (256): 	Total Chunks: 181, Chunks in use: 181. 45.2KiB allocated for chunks. 45.2KiB in use in bin. 912B client-requested in use in bin.
2024-03-07 20:07:42.341440: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (512): 	Total Chunks: 91, Chunks in use: 90. 47.8KiB allocated for chunks. 47.2KiB in use in bin. 38.4KiB client-requested in use in b

ResourceExhaustedError: Graph execution error:

Detected at node 'gradient_tape/model_8/output/MatMul/MatMul' defined at (most recent call last):
    File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/usr/local/lib/python3.8/dist-packages/ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "/usr/local/lib/python3.8/dist-packages/traitlets/config/application.py", line 1075, in launch_instance
      app.start()
    File "/usr/local/lib/python3.8/dist-packages/ipykernel/kernelapp.py", line 739, in start
      self.io_loop.start()
    File "/usr/local/lib/python3.8/dist-packages/tornado/platform/asyncio.py", line 205, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.8/asyncio/base_events.py", line 570, in run_forever
      self._run_once()
    File "/usr/lib/python